In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Weds Feb  2 13:20:25 2020
This script obtains web content.

@author: rachelcalder

https://creativeloafing.com/
Search wonderroot
"""

#import necessary packages
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import re

#This section grabs links off of the search pages
a_link = []
page_counter = []
z = 0 #82
pagecounter= 0
for i in range(82):
  url = f'https://creativeloafing.com/tiki-searchindex.php?filter%5Bcontent%5D=WonderRoot&&offset={z}'
  z += 25
  pagecounter += 1
  response = requests.get(url)
  link_base = 'https://creativeloafing.com/'
  content = response.content
        #restructure url data for readability
  soup = BeautifulSoup(content, "html.parser")
  
  headers = soup.find_all('strong')
  starter = 0
  link_num = 0
  for header in headers:
          link = header.find('a')
          if starter == 1:
            link_num += 1
            a_link.append(link_base + link.get('href'))
            page_counter.append(str(pagecounter) + '-' + str(link_num))
          if link.get('href') == "community":
            starter = 1

In [ ]:
#pulling page content
df = pd.DataFrame()
title = []
author = []
date = []
coverimg = []
pg_content = []
for i in a_link:
  response = requests.get(i)
  content = response.content
  soup = BeautifulSoup(content, "html.parser")
  if len(soup.find_all('title')) > 0:
    title.append(soup.find_all('title')[0].text)
  else:
    title.append('N/A')
  if len(soup.find_all("div", {"class": "byline clearfix"}))>0:
    author.append(soup.find_all("div", {"class": "byline clearfix"})[0].get_text('\n').split('\n')[1])
    date.append(soup.find_all("div", {"class": "byline clearfix"})[0].get_text('\n').split('\n')[3])
  else:
    author.append('N/A')
    date.append('N/A')
  if len(soup.find_all("div", {"class": "main-img"})) > 1:
    coverimg.append(soup.find_all("div", {"class": "main-img"})[1])
  else:
    coverimg.append('N/A')
  if len(soup.find_all("div", {"class": "col-sm-8"})) > 0:
    pg_content.append(soup.find_all("div", {"class": "col-sm-8"})[0].text)
  else:
    pg_content.append('N/A')

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [ ]:
#remove non text characters from body 
pg_content2 = []
for i in pg_content:
  pg_content2.append(i.replace('\n', '').replace('\t', '').replace('\r',''))
pg_content2

#Build final table
df = pd.DataFrame()
df['pageFound']=page_counter
df['link']=a_link
df['title']=title
df['author']=author
df['date']=date
df['coverimage']=coverimg
df['text']=pg_content2

In [ ]:
df.to_csv('creative_loafing_all.csv')

In [ ]:
#Original method we aren't goin to use
"""
df = pd.DataFrame()
title = []
author = []
date = []
text = []
used_links = []
unused_links = []
for i in a_link:
  response = requests.get(i)
  content = response.content
  soup = BeautifulSoup(content, "html.parser")

  pg_content = []
  if len(soup.find_all('article')) > 0:
    for k in soup.find_all('article')[0].get_text().split('\n'):
      if k != '':
        pg_content.append(k)
    if len(pg_content) > 1 and '\t' not in pg_content[0]:
      splitted = re.split('By | Monday |Mon| Tuesday |Tues| Wednesday |Weds| Thursday |Thurs| Friday |Fri| Saturday |Sat| Sunday |Sun',pg_content[1])
      if len(splitted) > 2:
        title.append(pg_content[0])
        author.append(splitted[1])
        date.append(splitted[2])
        text.append(pg_content[2])
        used_links.append(i)
      else:
        unused_links.append(i)
    else:
      unused_links.append(i)
  else:
      unused_links.append(i)
df['title']=title
df['link']=used_links
df['author']=author
df['date']=date
df['text']=text
df1 = pd.DataFrame()
df1['unused_links'] = unused_links
df1.to_csv("failed_links1.csv")
df.to_csv("round1_scrape.csv")
"""

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [2]:
#link testing

import requests
from bs4 import BeautifulSoup 
url = 'https://creativeloafing.com/content-153414-PHOTOS:-Stuck-Lucky-at-WonderRoot'
response = requests.get(url)
content = response.content
        #restructure url data for readability
soup = BeautifulSoup(content, "html.parser")

